In [1]:
%cd '/content/drive/MyDrive/Advanced Research Methods/Homework 2'

/content/drive/MyDrive/Advanced Research Methods/Homework 2


In [2]:
import pandas as pd
import random, math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import fftpack
import torch.utils.data as utils
import os
from pathlib import Path
from sklearn.metrics import mean_absolute_percentage_error
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [3]:
def reshape(root):

 tps_df = pd.read_pickle(root)

 reshaped_tps_df = pd.DataFrame()
 reshaped_tps_df['TIME'] = tps_df.time.unique()
 for seg in tps_df.segmentID.unique():
     column = tps_df[tps_df['segmentID'] == seg][['time','TrafficIndex_GP']].drop_duplicates(subset=['time'])
     column.columns = ['TIME', str(seg)]
     reshaped_tps_df = reshaped_tps_df.join(column.set_index('TIME'), on='TIME')

 reshaped_tps_df = reshaped_tps_df.set_index('TIME')
 return reshaped_tps_df



In [4]:
train_data = reshape('tps_df.pkl')


In [5]:
class TrafficForecast(Dataset):
    def __init__(self,df, window, horizon):
        self.window = window
        self.horizon = horizon
        self.df = df
        self.inputs = []
        self.targets = []

        # df = pd.read_pickle(self.pkl_path)
        # df['time'] = pd.to_datetime(df['time'])
        # df['unix_timestamp'] = df['time'].astype(int) / 10**9
        # df['dow'] = df['time'].dt.weekday
        # df['hour'] = df['time'].dt.hour
        # # df['min'] = df['time'].dt.minute
        # self.df = df.sample(frac=1).reset_index(drop=True)
        # self.seg_ids = self.df['segmentID'].unique()

        self.setup_forecast()

    def setup_forecast(self):
        for segid in self.df.columns:
          self.df = self.df.fillna(method="ffill")
          TI_series = self.df[segid].values

          for t in range(0,len(TI_series)-(self.window+self.horizon)):
              x = TI_series[t:t+self.window]
              y = TI_series[t+self.window:(t+self.window+self.horizon)]


              self.inputs.append(x)
              self.targets.append(y)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(self.window,1)
        y = torch.tensor(self.targets[idx],dtype=torch.float32)

        return {'inputs':X,'outputs':y}


In [6]:
tobj = TrafficForecast(train_data,36,12)


In [7]:
for idx, data in enumerate(tobj):
  print (data['inputs'].shape, data['outputs'].shape)
  if idx == 0:
    break


torch.Size([36, 1]) torch.Size([12])


In [8]:
bs = 200
dataloader_train = DataLoader(tobj, batch_size=bs, shuffle=False, num_workers=4,drop_last=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [9]:
from models import MyLSTM, MyGRU, MyRNNCell, MyRNN

In [10]:
model = MyLSTM(n_inputs=1, n_hidden=10, n_rnnlayers=2, n_outputs=12)
# model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
mean_loss = []
losses = []

n_epochs = 20

for it in tqdm(range(n_epochs)):
  # zero the parameter gradients
  for i_batch, sample_batched in enumerate(dataloader_train):
    optimizer.zero_grad()
    outputs = model(sample_batched['inputs'])
    loss = criterion(outputs, sample_batched['outputs'])
    losses.append(loss.item())
    loss.backward()
    optimizer.step()



  mean_loss.append(np.mean(losses))

  if (it+1) % 1 == 0:
    print(f'Epoch {it+1}/{n_epochs}, Training Loss: {np.mean(losses):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
  5%|▌         | 1/20 [

Epoch 1/20, Training Loss: 0.0179


 10%|█         | 2/20 [06:48<1:01:07, 203.73s/it]

Epoch 2/20, Training Loss: 0.0119


 15%|█▌        | 3/20 [10:14<58:08, 205.18s/it]  

...

Epoch 18/20, Training Loss: 0.0063


 95%|█████████▌| 19/20 [1:04:33<03:24, 204.61s/it]

Epoch 19/20, Training Loss: 0.0063


100%|██████████| 20/20 [1:07:53<00:00, 203.66s/it]

Epoch 20/20, Training Loss: 0.0062


In [12]:
torch.save(model.state_dict(), 'lstmmodel.pt')


In [13]:
class Test(Dataset):
    def __init__(self,df):

        self.df = df
        self.inputs = []

        self.setup_forecast()

    def setup_forecast(self):


        for segid in self.df.columns:
          self.df = self.df.fillna(method="ffill")
          TI_series = self.df[segid].values
          self.inputs.append(TI_series)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(-1,1)


        return {'inputs':X}

In [14]:
def run_files(i,tobj):

    pkl_file= (pkl_files[i])
    t_value = reshape(os.path.join('Test',pkl_file))
    tobj_ = Test(t_value)
    dfp = pd.DataFrame()
    for idx,data in enumerate(tobj_):
      input_data = data['inputs'].reshape(-1,36,1)
      preds = model(input_data).detach().cpu().numpy()[0]
      dfp[idx] = preds
    return dfp




In [39]:
pkl_files = os.listdir('Test')
print(pkl_files)

['tps_6.pkl', 'tps_9.pkl', 'tps_2.pkl', 'tps_14.pkl', 'tps_4.pkl', 'tps_5.pkl', 'tps_3.pkl', 'tps_12.pkl', 'tps_7.pkl', 'tps_13.pkl', 'tps_10.pkl', 'tps_1.pkl', 'tps_11.pkl', 'tps_15.pkl', 'tps_8.pkl']


In [41]:
op0 = run_files(0,tobj)
op0.index = pd.date_range(start='2020-06-07 11:15:00', end='2020-06-07 14:00:00', freq='15min').astype(int) / 10**9
op0.columns = train_data.columns

op1 = run_files(1,tobj)
op1.index = pd.date_range(start='2020-06-10 14:15:00', end='2020-06-10 17:00:00', freq='15min').astype(int) / 10**9
op1.columns = train_data.columns

op2 = run_files(2,tobj)
op2.index = pd.date_range(start='2020-06-03 7:15:00', end='2020-06-03 10:00:00', freq='15min').astype(int) / 10**9
op2.columns = train_data.columns

op3 = run_files(3,tobj)
op3.index = pd.date_range(start='2020-06-15 19:15:00', end='2020-06-15 22:00:00', freq='15min').astype(int) / 10**9
op3.columns = train_data.columns


op4 = run_files(4,tobj)
op4.index = pd.date_range(start='2020-06-05 9:15:00', end='2020-06-05 12:00:00', freq='15min').astype(int) / 10**9
op4.columns = train_data.columns

op5 = run_files(5,tobj)
op5.index = pd.date_range(start='2020-06-06 10:15:00', end='2020-06-06 13:00:00', freq='15min').astype(int) / 10**9
op5.columns = train_data.columns

op6 = run_files(6,tobj)
op6.index = pd.date_range(start='2020-06-04 8:15:00', end='2020-06-04 11:00:00', freq='15min').astype(int) / 10**9
op6.columns = train_data.columns

op7 = run_files(7,tobj)
op7.index = pd.date_range(start='2020-06-13 17:15:00', end='2020-06-13 20:00:00', freq='15min').astype(int) / 10**9
op7.columns = train_data.columns

op8 = run_files(8,tobj)
op8.index = pd.date_range(start='2020-06-08 12:15:00', end='2020-06-08 15:00:00', freq='15min').astype(int) / 10**9
op8.columns = train_data.columns

op9 = run_files(9,tobj)
op9.index = pd.date_range(start='2020-06-14 18:15:00', end='2020-06-14 21:00:00', freq='15min').astype(int) / 10**9
op9.columns = train_data.columns

op10 = run_files(10,tobj)
op10.index = pd.date_range(start='2020-06-11 15:15:00', end='2020-06-11 18:00:00', freq='15min').astype(int) / 10**9
op10.columns = train_data.columns

op11 = run_files(11,tobj)
op11.index = pd.date_range(start='2020-06-02 6:15:00', end='2020-06-02 9:00:00', freq='15min').astype(int) / 10**9
op11.columns = train_data.columns

op12 = run_files(12,tobj)
op12.index = pd.date_range(start='2020-06-12 16:15:00', end='2020-06-12 19:00:00', freq='15min').astype(int) / 10**9
op12.columns = train_data.columns

op13 = run_files(13,tobj)
op13.index = pd.date_range(start='2020-06-16 20:15:00', end='2020-06-16 23:00:00', freq='15min').astype(int) / 10**9
op13.columns = train_data.columns

op14 = run_files(14,tobj)
op14.index = pd.date_range(start='2020-06-09 13:15:00', end='2020-06-09 16:00:00', freq='15min').astype(int) / 10**9
op14.columns = train_data.columns


final_output = pd.concat([op11,op2,op6,op4,op5,op0,op8,op14,op1,op10,op12,op7,op9,op3,op13])

In [42]:
print(final_output.shape)

(180, 87)


In [43]:
print(final_output)

                    11        12        13        14        15        16  \
1.591078e+09  0.983004  0.978382  0.887817  0.903373  0.503576  0.809216   
1.591079e+09  0.983782  0.978983  0.894091  0.907317  0.533178  0.814624   
1.591080e+09  0.984087  0.979085  0.898062  0.909176  0.549242  0.816752   
1.591081e+09  0.984147  0.979090  0.903416  0.912302  0.576109  0.818708   
1.591082e+09  0.984203  0.979248  0.909141  0.916280  0.605440  0.823829   
...                ...       ...       ...       ...       ...       ...   
1.592345e+09  1.001498  1.001465  1.003834  1.002962  1.008433  1.030416   
1.592346e+09  1.001857  1.001832  1.004774  1.003529  1.009054  1.039880   
1.592347e+09  1.002129  1.002068  1.005028  1.003852  1.009921  1.043628   
1.592348e+09  1.002590  1.002585  1.006597  1.004636  1.010140  1.047604   
1.592348e+09  1.002720  1.002637  1.005775  1.004571  1.010516  1.046971   

                    17        18        19        20  ...        94        95  \
1.5910

In [46]:
final_output.to_json('final_output_lstm.json')

In [47]:
pd.read_json('final_output_lstm.json')

<ipython-input-47-0445f6108cb7>:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.read_json('final_output_lstm.json')


,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
2020-06-02 06:15:28,0.983004,0.978382,0.887817,0.903373,0.503576,0.809216,0.880805,0.996692,1.000498,0.992118,...,1.000477,1.000477,1.000477,1.000477,0.992979,1.000477,1.000477,1.000477,1.000477,1.000477
2020-06-02 06:30:24,0.983782,0.978983,0.894091,0.907317,0.533178,0.814624,0.868837,0.997499,1.000775,0.992702,...,1.000743,1.000743,1.000743,1.000743,0.993531,1.000743,1.000743,1.000743,1.000743,1.000743
2020-06-02 06:45:20,0.984087,0.979085,0.898062,0.909176,0.549242,0.816752,0.860221,0.997901,1.000776,0.992921,...,1.000738,1.000738,1.000738,1.000738,0.993728,1.000738,1.000738,1.000738,1.000738,1.000738
2020-06-02 07:00:16,0.984147,0.979090,0.903416,0.912302,0.576109,0.818708,0.867943,0.997748,1.000453,0.992765,...,1.000414,1.000414,1.000414,1.000414,0.993556,1.000414,1.000414,1.000414,1.000414,1.000414
2020-06-02 07:15:12,0.984203,0.979248,0.909141,0.916280,0.605440,0.823829,0.872940,0.997513,0.999971,0.992532,...,0.999926,0.999926,0.999926,0.999926,0.993297,0.999926,0.999926,0.999926,0.999926,0.999926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16 22:00:32,1.001498,1.001465,1.003834,1.002962,1.008433,1.030416,0.957875,0.952450,0.988242,0.994582,...,0.999218,1.008147,1.000095,0.999435,0.999901,0.999218,0.999218,1.000939,1.000277,0.999223
2020-06-16 22:15:28,1.001857,1.001832,1.004774,1.003529,1.009054,1.039880,0.961937,0.956152,0.989322,0.995615,...,0.999309,1.008816,1.000250,0.999545,1.000059,0.999309,0.999309,1.001159,1.000474,0.999309
2020-06-16 22:30:24,1.002129,1.002068,1.005028,1.003852,1.009921,1.043628,0.965925,0.959400,0.990169,0.996195,...,0.999432,1.008852,1.000399,0.999680,1.000219,0.999432,0.999432,1.001341,1.000658,0.999472
2020-06-16 22:45:20,1.002590,1.002585,1.006597,1.004636,1.010140,1.047604,0.968520,0.962171,0.991085,0.997412,...,0.999661,1.010211,1.000679,0.999921,1.000495,0.999661,0.999661,1.001682,1.000963,0.999644
